# Project 3: Astronomical Source Detection
## Eric Gonzalez Corona
-------------------------------------------
# Part 1 : SEP tutorial


## Start by importing numpy as per usual and, this time we will import SEP which helps us detect sources in an image and perform certain calculations

In [ ]:
import numpy as np
import sep

## Instead of using fitsio, I will use astropy which means some adjustments will have to 
## be made.
------------
## Will import fits from  astropy.io

In [ ]:
from astropy.io import fits # import fits from astropy.io which will allow us to open fits files
import matplotlib.pyplot as plt # import this to plot whatever we may need to plot
from matplotlib import rcParams # allows us to vary certain parameters that will be used later
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.] # Set the size: do it once here, and never have to do it again in later in the 
                                        #code

## Open the fits file, which was downloaded from SEP GitHub account

In [ ]:
fname = "image.fits" # already in same folder
hdu_list = fits.open(fname)
hdu_list.info()

## Image information typically in PRIMARY block, accessed by indexing hdu_list

In [ ]:
image_data = hdu_list["PRIMARY"].data

## Now we can close FITS file because stored as a variable

In [ ]:
hdu_list.close()

## Showing the Data

In [ ]:
# show the image
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('skyview.png' , bbox_inches="tight",dpi=600)

## Background subtraction time, taking away outliers, black is the outlier

In [ ]:
bkg = sep.Background(image_data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)


In [ ]:
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

## Show the Background, as well have save the figure

In [ ]:

plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('background.png' , bbox_inches="tight",dpi=600)

## Treat background noise as a 2-d array.

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('noise.png' , bbox_inches="tight",dpi=600)

##  Now we subtract the background

In [ ]:

# subtract the background
data_sub = image_data - bkg

## Now we do object detection

## note that the number "1.5" refers to a sigma value or threshold value

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)


len(objects) # how many objects were detected


from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), # gives us the coordinates
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig('tutorialfinal.png' , bbox_inches="tight",dpi=600) # saves the figure

## The objects function has several uses/information which can be extracted and displayed

In [ ]:
# available fields
objects.dtype.names

## Now we will show/calculate the fluxes of 9 objects in the image

In [ ]:

flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)


# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))